In [7]:
!nvidia-smi

Wed Feb 25 11:11:44 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             28W /   70W |    3241MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
import torch
torch.cuda.is_available()

True

In [9]:

import os

# System dependencies
!apt-get update -y
!apt-get install -y ffmpeg

# Core Python libraries
!pip install -q openai-whisper faster-whisper gdown

# Translation + Text processing
!pip install -q transformers sentencepiece

# Voice cloning (Coqui XTTS)
!pip install -q TTS

# Lip sync (Wav2Lip) and optional face restoration
if not os.path.exists("Wav2Lip"):
    !git clone https://github.com/Rudrabha/Wav2Lip.git
!pip install -r Wav2Lip/requirements.txt
!pip install -q gfpgan

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,383 kB]
Get:10 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,776 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted

In [10]:
!pip install gdown

In [11]:
!gdown --id 1aqolvo-SutfS9HIRL2LDP79pBOJdZPMI -O input.mp4

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:139: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1aqolvo-SutfS9HIRL2LDP79pBOJdZPMI
From (redirected): https://drive.google.com/uc?id=1aqolvo-SutfS9HIRL2LDP79pBOJdZPMI&confirm=t&uuid=b12e4ef0-925b-464d-8754-c1b20a178dd0
To: /content/input.mp4
100% 622M/622M [00:08<00:00, 70.1MB/s]


In [13]:
import os

def extract_chunk(input_path, output_path):
    os.system(f"ffmpeg -i {input_path} -ss 00:00:15 -to 00:00:30 -c copy {output_path}")

extract_chunk("input.mp4", "chunk.mp4")

In [14]:
def extract_audio(video_path, audio_path):
    os.system(f"ffmpeg -i {video_path} -q:a 0 -map a {audio_path}")

extract_audio("chunk.mp4", "audio.wav")

In [15]:
from faster_whisper import WhisperModel

def transcribe_auto(audio_path):
    # Changed compute_type to "float32" for broader compatibility
    model = WhisperModel("medium", compute_type="float32")

    segments, info = model.transcribe(
        audio_path,
        task="translate"   # Always translate to English
    )

    print("Detected language:", info.language)
    print("Language probability:", info.language_probability)

    full_text = ""
    for segment in segments:
        full_text += segment.text + " "

    return full_text.strip()

transcript_en = transcribe_auto("audio.wav")
print("\nEnglish Transcript:\n", transcript_en)

Detected language: kn
Language probability: 0.8742818832397461

English Transcript:
 So now lets see how to protect your hygiene  First step is to brush your teeth well before you go for a day's booking


In [16]:
!pip install -q transformers sentencepiece accelerate

In [17]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load model once (outside function ideally)
model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def translate_to_hindi(text):
    # NLLB language codes
    src_lang = "eng_Latn"
    tgt_lang = "hin_Deva"

    tokenizer.src_lang = src_lang

    inputs = tokenizer(text, return_tensors="pt", truncation=True)

    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
            max_length=512
        )

    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text


# Use your English transcript
hindi_text = translate_to_hindi(transcript_en)

print("Hindi Translation:\n")
print(hindi_text)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Hindi Translation:

तो अब आइए देखें कि अपनी स्वच्छता की रक्षा कैसे करें पहला कदम एक दिन के बुकिंग के लिए जाने से पहले अपने दांतों को अच्छी तरह से ब्रश करना है


In [18]:
!pip install edge-tts

In [19]:
import edge_tts
import asyncio

async def generate_hindi_speech(text, output_path="hindi_speech.wav"):
    voice = "hi-IN-SwaraNeural"
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(output_path)

# Removed asyncio.run() and used await directly
await generate_hindi_speech(hindi_text)

print("Hindi speech generated.")

Hindi speech generated.


In [20]:
!ls

audio.wav  chunk.mp4  hindi_speech.wav	input.mp4  sample_data	Wav2Lip


In [37]:
import librosa
import os

orig_duration = librosa.get_duration(path="audio.wav")
hindi_duration = librosa.get_duration(path="hindi_speech.wav")

print("Original:", orig_duration)
print("Hindi:", hindi_duration)

speed_factor = hindi_duration / orig_duration  # slow down Hindi

os.system(f"ffmpeg -y -i hindi_speech.wav -filter:a 'atempo={speed_factor}' adjusted_hindi.wav")

adjusted_duration = librosa.get_duration(path="adjusted_hindi.wav")
print("Adjusted:", adjusted_duration)

Original: 14.997333333333334
Hindi: 11.784
Adjusted: 14.966833333333334


In [63]:
!ffmpeg -y \
-i chunk.mp4 \
-i adjusted_hindi.wav \
-c:v copy \
-map 0:v:0 \
-map 1:a:0 \
-shortest \
final_output.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [64]:
!ls

adjusted_hindi.wav  chunk_019.mp4  chunk_036.mp4  chunk_053.mp4
audio.wav	    chunk_020.mp4  chunk_037.mp4  chunk_054.mp4
chunk_004.mp4	    chunk_021.mp4  chunk_038.mp4  chunk_055.mp4
chunk_005.mp4	    chunk_022.mp4  chunk_039.mp4  chunk_056.mp4
chunk_006.mp4	    chunk_023.mp4  chunk_040.mp4  chunk_057.mp4
chunk_007.mp4	    chunk_024.mp4  chunk_041.mp4  chunk_058.mp4
chunk_008.mp4	    chunk_025.mp4  chunk_042.mp4  chunk_059.mp4
chunk_009.mp4	    chunk_026.mp4  chunk_043.mp4  chunk_060.mp4
chunk_010.mp4	    chunk_027.mp4  chunk_044.mp4  chunk_061.mp4
chunk_011.mp4	    chunk_028.mp4  chunk_045.mp4  chunk.mp4
chunk_012.mp4	    chunk_029.mp4  chunk_046.mp4  clean_input.mp4
chunk_013.mp4	    chunk_030.mp4  chunk_047.mp4  final_output.mp4
chunk_014.mp4	    chunk_031.mp4  chunk_048.mp4  hindi_speech.wav
chunk_015.mp4	    chunk_032.mp4  chunk_049.mp4  input.mp4
chunk_016.mp4	    chunk_033.mp4  chunk_050.mp4  sample_data
chunk_017.mp4	    chunk_034.mp4  chunk_051.mp4  silent_video.mp4
chunk_018